Single Step Convolutional Nueral Network

# CNN Time Series Forecasting

In [29]:
#Importing Libraries
from numpy import array
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

In [30]:
import math 
import numpy as np 
import pandas as pd 
from datetime import date, timedelta, datetime

end_date =  date.today().strftime("%Y-%m-%d")
start_date = '2021-01-01'

stockname = 'Turkish Airlines'
symbol = 'THYAO.IS'

import yfinance as yf 
df = yf.download(symbol, start=start_date, end=end_date)

df.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-12-31 00:00:00+03:00,12.91,13.09,12.88,13.02,13.02,56720032
2021-01-04 00:00:00+03:00,12.97,12.97,12.67,12.68,12.68,59948726
2021-01-05 00:00:00+03:00,12.64,12.80,12.52,12.58,12.58,90905126
2021-01-06 00:00:00+03:00,12.73,12.81,12.63,12.68,12.68,63226286
2021-01-07 00:00:00+03:00,12.74,12.76,12.62,12.68,12.68,63579108


In [47]:
# split a univariate sequence into samples
def split_sequence(sequence, steps):
  X, y = [], []
  for start in range(len(sequence)):
    # define the end index of the sequence
    end_index = start + steps
    # to check if end_index stays in the allowable limit
    if end_index > len(sequence)-1:
      break
    # extract input and output parts of the sequence
    sequence_x, sequence_y = sequence[start : end_index], sequence[end_index]
    X.append(sequence_x)
    y.append(sequence_y)
  return array(X), array(y)

In [50]:
# choose a number of time steps
raw_sequence = df["Close"].tolist()
steps = 5

# split into samples
X, y = split_sequence(raw_sequence, steps)

print("**************************************************************")
print("The input raw sequence on which we will test our CNN model")
print(raw_sequence)
print("**************************************************************")
print("The features (X) ,lagged values of time series, for our CNN model")
print(X)
print("**************************************************************")
print("The respective observed values (y) of the sequence for training in CNN model")
print(y)
print("**************************************************************")

**************************************************************
The input raw sequence on which we will test our CNN model
[13.020000457763672, 12.680000305175781, 12.579999923706055, 12.680000305175781, 12.680000305175781, 12.84000015258789, 12.699999809265137, 12.850000381469727, 13.260000228881836, 13.239999771118164, 13.279999732971191, 13.239999771118164, 13.489999771118164, 13.260000228881836, 13.029999732971191, 12.930000305175781, 12.699999809265137, 12.630000114440918, 11.949999809265137, 12.329999923706055, 12.109999656677246, 12.010000228881836, 12.0600004196167, 12.149999618530273, 12.199999809265137, 12.15999984741211, 12.149999618530273, 12.170000076293945, 12.210000038146973, 12.149999618530273, 12.0, 12.399999618530273, 12.180000305175781, 12.199999809265137, 12.270000457763672, 12.300000190734863, 12.640000343322754, 13.020000457763672, 12.829999923706055, 13.390000343322754, 13.489999771118164, 13.800000190734863, 14.8100004196167, 14.920000076293945, 14.72000026702880

In [51]:
# reshape 2D input data to 3D input data
features = 1
X = X.reshape((X.shape[0], X.shape[1], features))

print("After reshaping , the shape of input X")
print(X.shape)
print("**************************************************************")
print("**************************************************************")
print("The final form of input feature matrix X")
print(X)
print("**************************************************************")
print("**************************************************************")
print("Feature matrix X is ready for input to CNN model.We have used feature engineering" 
    +"to convert a sequence to matrix with image matrix shape to find patterns in sequence")
print("**************************************************************")

After reshaping , the shape of input X
(522, 5, 1)
**************************************************************
**************************************************************
The final form of input feature matrix X
[[[ 13.02000046]
  [ 12.68000031]
  [ 12.57999992]
  [ 12.68000031]
  [ 12.68000031]]

 [[ 12.68000031]
  [ 12.57999992]
  [ 12.68000031]
  [ 12.68000031]
  [ 12.84000015]]

 [[ 12.57999992]
  [ 12.68000031]
  [ 12.68000031]
  [ 12.84000015]
  [ 12.69999981]]

 ...

 [[151.80000305]
  [150.19999695]
  [149.80000305]
  [144.80000305]
  [137.6000061 ]]

 [[150.19999695]
  [149.80000305]
  [144.80000305]
  [137.6000061 ]
  [130.80000305]]

 [[149.80000305]
  [144.80000305]
  [137.6000061 ]
  [130.80000305]
  [134.6000061 ]]]
**************************************************************
**************************************************************
Feature matrix X is ready for input to CNN model.We have used feature engineeringto convert a sequence to matrix with image matr

In [52]:
# Defining model Architecture
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(steps,
features)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [53]:
# fit model
model.fit(X, y, epochs=10000, verbose=0)

In [58]:
#Testing the Model
x_input = array([149.80000305,
  144.80000305,
  137.6000061 ,
  130.80000305,
  134.6000061 ])
x_input = x_input.reshape((1, steps, features))
y_pred = model.predict(x_input, verbose=0)
print("**************************************************************")
print(f"The next predicted value for the sequence {raw_sequence} is")
print(y_pred)
print("**************************************************************")


**************************************************************
The next predicted value for the sequence [13.020000457763672, 12.680000305175781, 12.579999923706055, 12.680000305175781, 12.680000305175781, 12.84000015258789, 12.699999809265137, 12.850000381469727, 13.260000228881836, 13.239999771118164, 13.279999732971191, 13.239999771118164, 13.489999771118164, 13.260000228881836, 13.029999732971191, 12.930000305175781, 12.699999809265137, 12.630000114440918, 11.949999809265137, 12.329999923706055, 12.109999656677246, 12.010000228881836, 12.0600004196167, 12.149999618530273, 12.199999809265137, 12.15999984741211, 12.149999618530273, 12.170000076293945, 12.210000038146973, 12.149999618530273, 12.0, 12.399999618530273, 12.180000305175781, 12.199999809265137, 12.270000457763672, 12.300000190734863, 12.640000343322754, 13.020000457763672, 12.829999923706055, 13.390000343322754, 13.489999771118164, 13.800000190734863, 14.8100004196167, 14.920000076293945, 14.720000267028809, 14.77999973297